<a href="https://colab.research.google.com/github/tabaahi07/Deep-Learning/blob/main/Implement_custom_neural_network_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv("insurance_data.csv")
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df[['age','affordibility']],df.bought_insurance,test_size=0.2 )

In [4]:
X_train_scaled = X_train.copy()
X_train_scaled['age'] = X_train_scaled['age'] / 100

X_test_scaled = X_test.copy()
X_test_scaled['age'] = X_test_scaled['age'] / 100

In [5]:
def sigmoid_numpy(X):
   return 1/(1+np.exp(-X))

sigmoid_numpy(np.array([12,0,1]))

array([0.99999386, 0.5       , 0.73105858])

In [6]:
def log_loss(y_true, y_predicted):
    epsilon = 1e-15
    y_predicted_new = [max(i,epsilon) for i in y_predicted]
    y_predicted_new = [min(i,1-epsilon) for i in y_predicted_new]
    y_predicted_new = np.array(y_predicted_new)
    return -np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new))

In [ ]:
model = keras.Sequential([
    keras.layers.Dense(1, input_shape=(2,), activation='sigmoid', kernel_initializer='ones', bias_initializer='zeros')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train_scaled, y_train, epochs=5000)

In [7]:
class myNN:
    def __init__(self):
        self.w1 = 1
        self.w2 = 1
        self.bias = 0

    def fit(self, X, y, epochs, loss_thresold):
        self.w1, self.w2, self.bias = self.gradient_descent(X['age'],X['affordibility'],y, epochs, loss_thresold)
        print(f"Final weights and bias: w1: {self.w1}, w2: {self.w2}, bias: {self.bias}")

    def predict(self, X_test):
        weighted_sum = self.w1*X_test['age'] + self.w2*X_test['affordibility'] + self.bias
        return sigmoid_numpy(weighted_sum)

    def gradient_descent(self, age,affordability, y_true, epochs, loss_thresold):
        w1 = w2 = 1
        bias = 0
        rate = 0.5
        n = len(age)
        for i in range(epochs):
            weighted_sum = w1 * age + w2 * affordability + bias
            y_predicted = sigmoid_numpy(weighted_sum)
            loss = log_loss(y_true, y_predicted)

            w1d = (1/n)*np.dot(np.transpose(age),(y_predicted-y_true))
            w2d = (1/n)*np.dot(np.transpose(affordability),(y_predicted-y_true))

            bias_d = np.mean(y_predicted-y_true)
            w1 = w1 - rate * w1d
            w2 = w2 - rate * w2d
            bias = bias - rate * bias_d

            if i%50==0:
                print (f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')

            if loss<=loss_thresold:
                print (f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')
                break

        return w1, w2, bias

In [8]:
customModel = myNN()
customModel.fit(X_train_scaled, y_train, epochs=8000, loss_thresold=0.4631)

Epoch:0, w1:0.9768596163995021, w2:0.9483247976327828, bias:-0.11202128979565554, loss:0.7087289107636217
Epoch:50, w1:1.5284843549603029, w2:1.083761708708183, bias:-1.2112120910113708, loss:0.5677761349597233
Epoch:100, w1:2.238417972403458, w2:1.2457187881510776, bias:-1.6216323265902792, loss:0.5395073347186082
Epoch:150, w1:2.9028189674842966, w2:1.2967644740414976, bias:-1.9293086134270543, loss:0.5178502904619189
Epoch:200, w1:3.516437428299769, w2:1.3065010568543074, bias:-2.184373994184789, loss:0.5001174151273571
Epoch:250, w1:4.080293051579788, w2:1.3017508288461255, bias:-2.407813695054942, loss:0.48535239464748636
Epoch:300, w1:4.597784500434267, w2:1.2930159519362188, bias:-2.609195991176252, loss:0.4729752392403471
Epoch:348, w1:5.0547754741511275, w2:1.284597972858603, bias:-2.7862741655270575, loss:0.46293309307889763
Final weights and bias: w1: 5.0547754741511275, w2: 1.284597972858603, bias: -2.7862741655270575


In [9]:
customModel.predict(X_test_scaled)
# We are getting almost same result by custom model as predicted by the keras model

5     0.790675
25    0.773451
19    0.356218
17    0.806916
10    0.356218
3     0.460634
dtype: float64

In [13]:
model.predict(X_test_scaled)

1/1 [==============================] - 0s 75ms/step


array([[0.79485685],
       [0.7772309 ],
       [0.34575793],
       [0.8114266 ],
       [0.34575793],
       [0.4773913 ]], dtype=float32)